In [1]:
# coding=utf-8
# https://www.akshare.xyz/demo.html#id1
from datetime import datetime

import backtrader as bt  # 升级到最新版
import matplotlib.pyplot as plt  # 由于 Backtrader 的问题，此处要求 pip install matplotlib==3.2.2
import akshare as ak  # 升级到最新版
import pandas as pd

plt.rcParams["font.sans-serif"] = ["SimHei"]
plt.rcParams["axes.unicode_minus"] = False

In [3]:
# 利用 AKShare 获取股票的后复权数据，这里只获取前 6 列
stock_hfq_df = ak.stock_zh_a_hist(symbol="000001", adjust="hfq").iloc[:, :6]
# 处理字段命名，以符合 Backtrader 的要求
stock_hfq_df.columns = [
    'date',
    'open',
    'close',
    'high',
    'low',
    'volume',
]
stock_hfq_df

,date,open,close,high,low,volume
0,1991-04-03,49.00,49.00,49.00,49.00,1
1,1991-04-04,48.76,48.76,48.76,48.76,3
2,1991-04-05,48.52,48.52,48.52,48.52,2
3,1991-04-06,48.28,48.28,48.28,48.28,7
4,1991-04-08,48.04,48.04,48.04,48.04,2
...,...,...,...,...,...,...
7305,2021-11-15,3151.23,3164.23,3196.74,3126.85,655090
7306,2021-11-16,3152.85,3130.10,3182.11,3121.97,601110
7307,2021-11-17,3118.72,3112.22,3143.10,3091.09,664640
7308,2021-11-18,3108.97,3061.84,3113.85,3050.46,799844


In [9]:
# 保存数据
stock_hfq_df.to_csv('./data/stock_hfq_df.txt')
type(stock_hfq_df)

pandas.core.frame.DataFrame

In [10]:


# 把 date 作为日期索引，以符合 Backtrader 的要求
stock_hfq_df.index = pd.to_datetime(stock_hfq_df['date'])

In [11]:
class MyStrategy(bt.Strategy):
    """
    主策略程序
    """
    params = (("maperiod", 20),)  # 全局设定交易策略的参数

    def __init__(self):
        """
        初始化函数
        """
        self.data_close = self.datas[0].close  # 指定价格序列
        # 初始化交易指令、买卖价格和手续费
        self.order = None
        self.buy_price = None
        self.buy_comm = None
        # 添加移动均线指标
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod
        )

    def next(self):
        """
        执行逻辑
        """
        if self.order:  # 检查是否有指令等待执行,
            return
        # 检查是否持仓
        if not self.position:  # 没有持仓
            if self.data_close[0] > self.sma[0]:  # 执行买入条件判断：收盘价格上涨突破20日均线
                self.order = self.buy(size=100)  # 执行买入
        else:
            if self.data_close[0] < self.sma[0]:  # 执行卖出条件判断：收盘价格跌破20日均线
                self.order = self.sell(size=100)  # 执行卖出

In [12]:
cerebro = bt.Cerebro()  # 初始化回测系统
start_date = datetime(1991, 4, 3)  # 回测开始时间
end_date = datetime(2020, 6, 16)  # 回测结束时间
data = bt.feeds.PandasData(dataname=stock_hfq_df, fromdate=start_date, todate=end_date)  # 加载数据
cerebro.adddata(data)  # 将数据传入回测系统
cerebro.addstrategy(MyStrategy)  # 将交易策略加载到回测系统中
start_cash = 1000000
cerebro.broker.setcash(start_cash)  # 设置初始资本为 100000
cerebro.broker.setcommission(commission=0.002)  # 设置交易手续费为 0.2%
cerebro.run()  # 运行回测系统

port_value = cerebro.broker.getvalue()  # 获取回测结束后的总资金
pnl = port_value - start_cash  # 盈亏统计

print(f"初始资金: {start_cash}\n回测期间：{start_date.strftime('%Y%m%d')}:{end_date.strftime('%Y%m%d')}")
print(f"总资金: {round(port_value, 2)}")
print(f"净收益: {round(pnl, 2)}")

cerebro.plot(style='candlestick')  # 画图

初始资金: 1000000
回测期间：19910403:20200616
总资金: 1217360.83
净收益: 217360.83


ImportError: cannot import name 'warnings' from 'matplotlib.dates' (C:\ProgramData\Anaconda3\envs\py39\lib\site-packages\matplotlib\dates.py)